In [1]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal
import pandas as pd

# **Partie 2**

## **Fonctions generiques**

In [2]:
signal_dict = np.load('apprentissage/a_1.npz') # signal_dict est un dictionnaire
#contenant les données contenues dans le fichier signal.npz
s = signal_dict['s'] # extraction du signal utile
t = signal_dict['t'] # extraction du vecteur temps associé au signal

px.line(x=t, y=s, labels={"x":"Temps (s)", "y":"Amplitude"},
title="a_1.npz").show()

#help(san.trans_fourier)
nu, S = san.trans_fourier(s, t)

S = S - np.mean(S)

S = np.absolute(S)
S = np.log10(S)
S = S / np.max(S) #normalisation de l'amplitude
print(np.max(S))

px.line(x=nu, y=S, labels={"x":"Temps (HZ)", "y":"Amplitude"},
title="TF a_1.npz").show()

1.0


## **analyse des groupes sonores**

## Procédure des boucles <br>
¨¨¨¨
**<h3>boucle lecture de** *lettres* <br>
¨¨¨¨**boucle lecture des courbes** *1 à 5* <br>
¨¨¨¨¨¨¨¨*san.detect_fondamentale* <br>
¨¨¨¨¨¨¨¨**boucle lecture des harmoniques et amplitudes** <br>
¨¨¨¨¨¨¨¨¨¨¨¨*san.detect_pics* <br>
¨¨¨¨¨¨¨¨¨¨¨¨*Moyenne* <br>
¨¨¨¨¨¨¨¨*Vecteurs de fréquences et amplitudes* <br>
¨¨¨¨¨¨¨¨*Ajout de la tranformée dans le tracé* <br>
¨¨¨¨**boucle parcour des harmoniques** *1 à 5*<br>
¨¨¨¨¨¨¨¨**boucle parcour des vecteurs** <br>
¨¨¨¨¨¨¨¨¨¨¨¨*calcul de la somme des distances pour Ecart-Type* <br>
¨¨¨¨¨¨¨¨*calcul Ecarts types pour chaque harmonique* <br>
</h3>

In [ ]:
LETTRES = ["a","e","i","o","u"]
DATA_vect_Freq = {"a":[0,0,0,0,0],"e":[0,0,0,0,0],"i":[0,0,0,0,0],"o":[0,0,0,0,0],"u":[0,0,0,0,0]} #contient le vecteur frequence associé à chaque spectre 1 à 5
DATA_vect_Amp = {"a":[0,0,0,0,0],"e":[0,0,0,0,0],"i":[0,0,0,0,0],"o":[0,0,0,0,0],"u":[0,0,0,0,0]} #contient le vecteur amplitude associé à chaque spectre 1 à 5
DATA_moyennes = {"a":[0,0],"e":[0,0],"i":[0,0],"o":[0,0],"u":[0,0]} ## [frequences moyennes, amplitudes moyennes]
DATA_ecartTypes = {"a":[0,0],"e":[0,0],"i":[0,0],"o":[0,0],"u":[0,0]} ## [écart type frequences, écart type amplitude]
H = 13  #nombre d'harmoniques

for var_lettre in LETTRES:
    courbes = [] #tranformee de Fourier complète des 5 signaux d'un son
    for var_indice in range (1,6): #varie de 1 à 5
        nom = var_lettre +"_" +str(var_indice) + ".npz" 
        signal_dict = np.load('apprentissage/'+nom) #ouvrir le fichier signal
        
        s = signal_dict['s'] # extraction du signal temporel
        t = signal_dict['t'] # extraction du vecteur temps associé au signal

        nu, S = san.trans_fourier(s, t) #transformee de Fourier
        S = S - np.mean(S)
        S = np.absolute(S)
        fondamentale = (san.detect_fondamentale(S, nu, 0.7*np.max(S))) #on relève la fondamentale tant que ça marche
        S = np.log10(S) #utilisation de l'echelle logarithmique
        S = S / np.max(S)  #normalisation de l'amplitude
        courbes.append(S)

        """A FAIRE : ajouter un moyen de determine le nb d harmoniques a prendre et de creer la longeur de liste automatiquement"""

        temp_freqs_harmoniques = [fondamentale]
        for var_rang_harmonique in range (H): #ajout des harmoniques suivantes
            temp_freqs_harmoniques.append((var_rang_harmonique+1)*fondamentale)
        #A partir d'ici on fait directement la moyenne des fondamentales et harmoniques
        
        SIGNAL_Harmonique_Freq, SIGNAL_Harmonique_Amp= san.detect_pics(S,temp_freqs_harmoniques,nu) #tuple (freq pic, amplitude pic)
        
        DATA_vect_Freq[var_lettre][var_indice-1] = SIGNAL_Harmonique_Freq
        DATA_vect_Amp[var_lettre][var_indice-1] = SIGNAL_Harmonique_Amp
        
        DATA_moyennes[var_lettre][0] += 1/5*SIGNAL_Harmonique_Freq
        DATA_moyennes[var_lettre][1] += 1/5*SIGNAL_Harmonique_Amp
        
    temp_vect_harmoniques_Freq, temp_vect_harmoniques_Amp = np.zeros(H + 1), np.zeros(H + 1)
        
    for var_rang_harmonique2 in range(H+1):#boucle des harmoniques
        '''print((DATA_vect_Freq[var_lettre]))'''
        temp_sommeMesuresFreq, temp_sommeMesuresAmp= 0, 0
        temp_ecartType_Freq, temp_ecartType_Amp = 0, 0
        for var_indice2 in range (5): #boucle des signaux
            temp_sommeMesuresFreq += (DATA_vect_Freq[var_lettre][var_indice2][var_rang_harmonique2] - DATA_moyennes[var_lettre][0][var_rang_harmonique2])**2
            temp_sommeMesuresAmp += (DATA_vect_Amp[var_lettre][var_indice2][var_rang_harmonique2] - DATA_moyennes[var_lettre][1][var_rang_harmonique2])**2
            #print("signal ",var_indice2, ". harmonique ", var_rang_harmonique2)
        """Calcul des Ecarts Type en fréquence et amplitude pour chaque Fondamentale + Harmonique"""
        temp_ecartType_Freq = np.sqrt((1/5)*temp_sommeMesuresFreq)
        temp_ecartType_Amp = np.sqrt((1/5)*temp_sommeMesuresAmp)

        #print("temp_vect_harmoniques_Freq[var_indice2][var_rang_harmonique2] ", temp_vect_harmoniques_Freq[var_indice2])
        #print("temp_ecartType_Freq ",temp_ecartType_Freq)
        
        temp_vect_harmoniques_Freq[var_rang_harmonique2] = temp_ecartType_Freq
        #print(temp_vect_harmoniques_Freq)
        temp_vect_harmoniques_Amp[var_rang_harmonique2] = temp_ecartType_Amp
        DATA_ecartTypes[var_lettre][0] = temp_vect_harmoniques_Freq
        #print("ecart type ", DATA_ecartTypes)
        DATA_ecartTypes[var_lettre][1] = temp_vect_harmoniques_Amp

    """Tracés des courbes"""
    COURBE_superposition = px.line(x=DATA_moyennes[var_lettre][0], y=DATA_moyennes[var_lettre][1], labels={"x":"", "y":"Amplitude"},
        title=var_lettre, markers='markers', range_x=[0, 5000], range_y=[-1.15,1.15]).show() #trace du vecteur moyen
    
    df_signaux = pd.DataFrame({
    "Temps (Hz)": np.concatenate([nu, nu, nu, nu, nu]),
    "Valeur": np.concatenate([courbes[0], courbes[1], courbes[2], courbes[3], courbes[4]]),
    "Signal": ["S1"]*len(nu) + ["S2"]*len(nu) + ["S3"]*len(nu) + ["S4"]*len(nu) + ["S5"]*len(nu)
    })
    px.line(df_signaux, x="Temps (Hz)", y="Valeur", color="Signal"
    ,title=("superposition des courbes "+var_lettre)).show() #trace des signaux superposes

    


## *à faire*

Mesurer l'écart type entre les différentes fréquences et amplitudes lors des test **Ecart Type**
et ainsi comparer l'écart de la valeur mesurée avec la moyenne des signaux testés **Ecart de Mesure**
afin de déterminer si l'**Ecart de Mesure** est inférieur à l'**Ecart Type**

## **Fonction de comparaison du signal avec le dictionnaire**

In [4]:
'''
A 5 harmoniques
E 3 harmoniques
I 4 harmoniques
O 3 harmoniques
U 5 harmoniques
'''

##COMPARAISON

signal_dict = np.load('test/e_6.npz') # signal_dict est un dictionnaire
#contenant les données contenues dans le fichier signal.npz
s = signal_dict['s'] # extraction du signal utile
t = signal_dict['t'] # extraction du vecteur temps associé au signal

#px.line(x=t, y=s, labels={"x":"Temps (s)", "y":"Amplitude"},title="a_1.npz").show()

nu, S = san.trans_fourier(s, t) #transformee de Fourier
S = S - np.mean(S)
S = np.absolute(S)
fondamentale = (san.detect_fondamentale(S, nu, 0.7*np.max(S))) #on relève la fondamentale tant que ça marche
S = np.log10(S) #utilisation de l'echelle logarithmique
S = S / np.max(S)  #normalisation de l'amplitude


freqs_harmoniques = [fondamentale]
for l in range (H): #ajout des harmoniques suivantes
    freqs_harmoniques.append((l + 1)*fondamentale)

ETUDE_Harmonique_Freq, ETUDE_Harmonique_Amp= san.detect_pics(S,freqs_harmoniques,nu)

ecartFreq = {"a":np.array([]),"e":0,"i":0,"o":0,"u":0} #contient des listes avec les ecarts de chaque pic avec la moyenne
ecartAmp = {"a": [],"e":0,"i":0,"o":0,"u":0}
ecartFmin = 1000
ecarAmin = 1
Juste_Lettre = "0"
px.line(x=nu, y=S, labels={"x":"Temps (HZ)", "y":"Amplitude"},
title="TF signal Etudié.npz").show()

for var_indice_lettre,var_lettre2 in enumerate(LETTRES) : #on parcoure chaque lettre
    print(var_lettre2)
    temp_Freq, temp_Amp = [], []
    for j in range (H + 1): #on parcoure les harmoniques du signal
        print("ETUDE_Harmonique_Freq",ETUDE_Harmonique_Freq[j])
        print("DATA_moyennes",DATA_moyennes[var_lettre2][0][j])
        temp_Freq.append(int(np.abs(ETUDE_Harmonique_Freq[j] - DATA_moyennes[var_lettre2][0][j]) * 1000))
        temp_Amp.append(int(np.abs(ETUDE_Harmonique_Amp[j] - DATA_moyennes[var_lettre2][1][j]) * 1000))
    
    for var_intermediaire in range(H + 1): #on reduit le nombre de décimales
        temp_Freq[var_intermediaire] = temp_Freq[var_intermediaire]/1000
        temp_Amp[var_intermediaire] = temp_Amp[var_intermediaire]/1000

    ecartFreq[var_lettre2] = temp_Freq
    ecartAmp[var_lettre2] = temp_Amp
    print('ecartFreq', ecartFreq[var_lettre2])
    print('DATA_ecartTypes', DATA_ecartTypes[var_lettre2][0])
    print('ecartAmp', ecartAmp[var_lettre2])
    print('DATA_ecartTypes', DATA_ecartTypes[var_lettre2][1])
    ''' ecart type
    temp_Freq.append(int(np.abs(ETUDE_Harmonique_Freq[j] - DATA_ecartTypes[var_lettre2][0][j]) * 1000))
    temp_Amp.append(int(np.abs(ETUDE_Harmonique_Amp[j] - DATA_ecartTypes[var_lettre2][1][j]) * 1000))
    '''
    ecartAMoyen = np.mean(ecartAmp[var_lettre2][j])
    ecartFMoyen = np.mean(ecartFreq[var_lettre2][j])
    '''print('ecartAMoyen','lettre',var_lettre2, ecartAMoyen)
    print('ecartFMoyen','lettre',var_lettre2, ecartFMoyen)'''
    if ecartAMoyen < ecarAmin:
        #ecartFmin = ecartFMoyen
        ecarAmin = ecartAMoyen
        Juste_Lettre = var_lettre2
print("la lettre qui correspond le mieux à nos données est : ", Juste_Lettre)

a
ETUDE_Harmonique_Freq 122.9795
DATA_moyennes 112.7812
ETUDE_Harmonique_Freq 243.95933333333332
DATA_moyennes 225.76236666666668
ETUDE_Harmonique_Freq 364.93916666666667
DATA_moyennes 340.5432333333333
ETUDE_Harmonique_Freq 487.91866666666664
DATA_moyennes 452.7245333333334
ETUDE_Harmonique_Freq 607.8986666666667
DATA_moyennes 563.7060333333334
ETUDE_Harmonique_Freq 732.8778333333333
DATA_moyennes 677.0871333333333
ecartFreq [10.198, 18.196, 24.395, 35.194, 44.192, 55.79]
DATA_ecartTypes [ 3.48653797  7.19324323 10.36359926 14.74067681 18.60227467 22.49425024]
ecartAmp [0.111, 0.01, 0.091, 0.19, 0.191, 0.296]
DATA_ecartTypes [0.01160329 0.01937446 0.03845819 0.07493371 0.15804784 0.09520546]
e
ETUDE_Harmonique_Freq 122.9795
DATA_moyennes 119.78003333333334
ETUDE_Harmonique_Freq 243.95933333333332
DATA_moyennes 239.56006666666667
ETUDE_Harmonique_Freq 364.93916666666667
DATA_moyennes 358.94016666666664
ETUDE_Harmonique_Freq 487.91866666666664
DATA_moyennes 479.12013333333334
ETUDE_Harm